In [15]:
import pandas as pd
import numpy as np


# 1 Numerisation des donnees

Creation d'un tableau des donees du fichier student-por.csv

In [16]:
def load_data_from_csv(csv_path:str):
    data = pd.read_csv(csv_path)
    return data

## Encodage des donnes

- [x]1 school (GP = 1 , MS = 2)
- [x]2 sex (F = 1 , M = 2)
- [x]4 adress (U = 1 , R = 2) 
- [x]5 famsize (LE3 = 1 , GT3 = 2)
- [x]6 Pstatus (T = 1 , A = 2)
- [x]9 Mjob & 10 Fjob ("teacher" = 1, 
                    "health" = 2,
                    "services" = 3,
                    "at_home" = 4,
                    "other" = 5)
- [x]11 reason ("home" = 1,
            "reputation" = 2,
            "course" = 3,
            "other" = 4)
- [x]12 guardian ("mother" = 1,
                "father" = 2,
                "other" = 3)
- [x]16 schoolsup (yes=1,no=2)
- [x]17 famsup (yes=1,no=2)
- [x]18 paid (yes=1,no=2)
- [x]19 activities (yes=1,no=2)
- [x]20 nursery (yes=1,no=2)
- [x]21 higher (yes=1,no=2)
- [x]22 internet (yes=1,no=2)
- [x]23 romantic (yes=1,no=2)


Fonction d'encodage de donnee

In [17]:
''' Permet de faire l'encodage des donnees 
    ARGS
        - data : la liste des donnees
        - indexes : la listes des colonnes a encoder
        - encode_rules : un dictionaire des regles a suivre pour encodes la valeur des donnees
'''
def encodage(data,indexes:list,encode_rules):
    for i in range(len(data)):
        for index in indexes:
            value = data.loc[i,index]
            try:
                value = encode_rules[index][value]
            except :
                value = encode_rules[value]
            data.loc[i,index] = value

In [18]:
# encodage(data=students,
#          indexes=["school",
#                   "sex",
#                   "address",
#                   "famsize",
#                   "Pstatus",
#                   "Mjob",
#                   "Fjob",
#                   "reason",
#                   "guardian",
#                   "schoolsup",
#                   "famsup",
#                   "paid",
#                   "activities",
#                   "nursery",
#                   "higher",
#                   "internet",
#                   "romantic"
#                 ],
#          encode_rules={ 'GP':0 , 'MS':1,# school
#                         'F':0 , 'M':1,  # sex
#                         'U':0 , 'R':1,   # address
#                         'LE3':0 ,'GT3':1, # famsize
#                         'T':0 , 'A':1,  # Pstatus
#                         'teacher':0,'health':1,'services':2,'at_home':3,
#                         'Mjob':{'other':4}, # Mjob
#                         'Fjob':{'other':4}, # Fjob
#                         "home":0,"reputation":1,"course":2,
#                         "reason":{"other":3},   #reason
#                         "guardian": {"mother":0,"father":1,"other":2},  # guardian
#                         'yes':0,'no':1 
#                     }
#         )

# 2 - Coefficient de correlation


r = sum((xi - x)(yi - y)) / sqrt( sum((xi - x)^2) * sum((yi - y)^2) )


In [19]:
''' Cree un tableau des coefficient de correlations entre 2 ligne de donnee '''
def cor_coeff(data,index_x,index_y):
    coffs = None
    # Recuperation des donnees des colonnes
    x_values = data[index_x]
    y_values = data[index_y]
    # Calculer la moyenne x_bar
    x_bar = np.mean(x_values)
    # Calculer la moyenne y_bar
    y_bar = np.mean(y_values)

    numerateur = 0
    d_sum1 = 0
    d_sum2 = 0

    for i in range(len(data)):
        calc_x = x_values[i] - x_bar # xi - x
        calc_y = y_values[i] - y_bar # yi - y

        numerateur += (calc_x*calc_y)
        d_sum1 += (calc_x)**2
        d_sum2 += (calc_y)**2

    return numerateur / np.sqrt((d_sum1)*(d_sum2))


In [20]:
''' Calcul les coefficients de corelations pour tous les collumns du data '''
def cors_coeffs(data):
    indexes = data.columns
    corr = dict()
    for x_index in indexes:
        value = dict()
        for y_index in indexes:
            value.__setitem__(y_index,cor_coeff(data,x_index,y_index))
        corr.__setitem__(x_index,value)
    return pd.DataFrame(corr) 


Analyser les coefficients de corelation pour decider des colonnes a garder

# 3 - Regression lineaire

Le but est de trouver une relation explicative entre une colonne Y et le reste des variales X

## Modelisation du probleme

Tena tanjona mahazo droite anakiray otriotrinzao :
$${
    y_i = a_0 + a_1 X_{i,1} + a_2 X_{i,2} + . . . + a_p X_{i,p} + ε_i ,\space\space
i = 1, . . . , n
}$$
Fa probleme anzay droite anakiray tokony mampety ny ligne de donees rehetra zay mety misy erreur

Supposons hananana valeur a0,...,ap donc hananana le droite

Rah tsy hanana le droite de inona tanjona?
Ahoana ny isafidianana anle droite tsara , minimiser le plus possible les erreurs soit faire en sorte que la somme des erreurs soit egale a 0 en choisissant les estimateurs b = a_chapeau
$${
    t_i = b_0 + b_1 X_{i,1} + b_2 X_{i,2} + . . . + b_p X_{i,p} + ε_i ,\space\space
i = 1, . . . , n
}$$

enfaite l'erreur peut etre formaliser comme tel :

$${ ε_i ≡ y_i − ŷ_i }$$

et donc minimiser cette erreur revient a faire son optimisation

Apres certaines demonstration on arrive a :
 



### Calcul des estimateurs de moindre carre

$${A = (X^TX)^{-1}  X^TY}$$

In [21]:
''' Determiner les coefficients de la droite de regression 
entre des donnees explicatifs et des donnees a expliquer avec
A = inv(Xt.X).Xt.Y
    ARGS 
        X : les valeurs explicatifs
        Y : les valeurs a expliques
    RETURN
        A les coefficients de la droite de regression
'''
def get_coeff_lineaire(x:pd.DataFrame,y:pd.DataFrame):
    # Calcul de la transposer de x
    xt = np.transpose(x)
    # Calcul de Xt.X
    xt_x = xt.dot(x)
    # Calcul de inv(Xt.X)
    xt_x = xt_x.astype(float)
    inv = np.linalg.inv(xt_x.values)
    inverse = pd.DataFrame(inv,xt_x.columns,xt_x.index)
    # Calcul de Xt.Y
    xt_y = xt.dot(y)
    # Calucul inv(Xt.X).Xt.Y
    a = inverse.dot(xt_y)
    return a

''' Determiner les coefficients de la droite de regresseion pour une colonne a expliquer specifier 
    ARGS    
        data
'''
def reg_model(data:pd.DataFrame , index):
    X = data.drop(index, axis='columns')
    # Création d'une colonne de 1s
    colonne_de_1s = np.ones((X.shape[0],1))
    # Ajout de la colonne de 1s à la matrice X
    X = pd.DataFrame(np.hstack((colonne_de_1s, X)))
    Y = pd.DataFrame(data[index])
    model= get_coeff_lineaire(X,Y)
    return model

### Calcul du coefficient de determination

- [x] calcul de  scr
    - scr = sum (yi - yi_ )^2
- [x] calcul de sce
    - sce = sum((yi)^2 - y_bar)
- [x] calcul de sct = sce + scr
- [x] calcul de r2 = sce/sct 

In [22]:
''' Permet de faire l'estimation ou prediction d'une colonne avec un model de regression '''
def get_estimation_lineaire(data:pd.DataFrame,index,model):
    estimations = list()
    # Les coefficents estimateur pour la doite 
    coeffs = model
    X = data.drop(index, axis='columns')
    for i in range(len(data)):
        yi = coeffs[index][0]
        j = 1
        for j_index in X.columns:
            xij =  data[j_index][i]
            yi += coeffs[index][j]*xij
            j+=1
        estimations.append(yi)
    
    return pd.DataFrame({index:estimations})

In [23]:
''' Determine le coefficient de determination pour connaitre la force du pouvoir predictif du model'''
def det_coeff(data:pd.DataFrame,index,model):
    Y = pd.DataFrame(data[index])
    # Creation de la matrice y des valeurs estimees
    y = get_estimation_lineaire(data,index,model)
    # Calcul des sommes
    scr = 0
    sce = 0
    y_bar = np.mean(Y.values)
    for i in range(len(data)):
        # Calcul de scr
        scr += (Y[index][i] - y[index][i])**2
        # Calcul de sce
        sce += ((y[index][i])**2 - y_bar)
    sct = scr + sce
    
    r2 = sce/sct
    return r2
# det_coeff(students,"G3")

# 4 - Test

Cree une fonction qui teste la variance des erreurs des donnees

f(data , pourcentage , cible)

- [x] separer les donnee
    - [x] shuffle le data
    - [x] utiliser les  pourcentage data pour le coeff de determination
    - [x] utiliser le reste pour determiner les coefficients de corelations
- [x] toruver les coeffs de corelation
- [x] eliminer les colonnes qui sont pas tres en corelation
- [x] trouver le coeff de deterination

In [24]:
''' Analyse le tableau des coefficients de corelation et determine a partir de cor_min les variables a 
    garder pour un index specifier
    ARGS
        data : la liste des donnees
        cor_coeffs : les coefficients de corelations
        index : le nom du colonne / variable de reference
        cor_min : le barem a suivre >=(+/-)corcor_min
    RETURN
        un tableau des donnees avec uniquement les variables colineaires entre-elles
'''
def cor_indexes( data:pd.DataFrame , cor_coeffs:pd.DataFrame , index:str , cor_min:float):
    # Recuperation des coefficients pour l'Index
    coeffs = cor_coeffs[index]
    # Cree une variable d'iteration des resultats
    result = data
    # Pour tout les indexes possibles on compare sa coefficent de corelastion
    # coeff <cor_min < 
    for variable in data.columns:
        coeff = cor_coeffs[index][variable]
        if ((coeff > -cor_min) and (coeff < cor_min) ):
            result = result.drop(variable,axis='columns')
    return result 


In [25]:
def data_sum_error(data:pd.DataFrame , index:str , model):
    Y = pd.DataFrame(data[index])
    y = get_estimation_lineaire(data,index,model)

    result = 0
    for i in range(len(data)):
        result += (Y[index][i] - y[index][i])**2
    return result

In [31]:
'''Fonction de test la qualite de la regression'''
def test(data:pd.DataFrame , pourcentage:float , cible:str ):
    # Separation des donnees
    shuffled_data   = data.sample(frac=1).reset_index(drop=True)
    # Calcul du nombre de lignes à conserver pour l'échantillon
    nombre_lignes_x = int(len(shuffled_data) * pourcentage)
    # Sélection des premières lignes pour former l'échantillon
    test_data = shuffled_data.iloc[:nombre_lignes_x]
    # Le reste des lignes forme l'échantillon de test
    coeff_data = shuffled_data.iloc[nombre_lignes_x:].reset_index(drop=True)
    # Determiner les coefficients de corelation selon les donees selectionnees
    corelation_coeffs = cors_coeffs(coeff_data)
    # Garder que les colonnes en corelation avec la cibl
    test_data = cor_indexes(data=test_data,
                            cor_coeffs=corelation_coeffs,
                            index=cible,
                            cor_min=0.1)
    model = reg_model(test_data,cible)
    print("MODEL \n",model)
    residu_sum = data_sum_error(test_data,cible,model) 
    return residu_sum


students = load_data_from_csv("3_donnees_student/student-por.csv")
encodage(data=students,
         indexes=["school",
                  "sex",
                  "address",
                  "famsize",
                  "Pstatus",
                  "Mjob",
                  "Fjob",
                  "reason",
                  "guardian",
                  "schoolsup",
                  "famsup",
                  "paid",
                  "activities",
                  "nursery",
                  "higher",
                  "internet",
                  "romantic"
                ],
         encode_rules={ 'GP':0 , 'MS':1,# school
                        'F':0 , 'M':1,  # sex
                        'U':0 , 'R':1,   # address
                        'LE3':0 ,'GT3':1, # famsize
                        'T':0 , 'A':1,  # Pstatus
                        'teacher':0,'health':1,'services':2,'at_home':3,
                        'Mjob':{'other':4}, # Mjob
                        'Fjob':{'other':4}, # Fjob
                        "home":0,"reputation":1,"course":2,
                        "reason":{"other":3},   #reason
                        "guardian": {"mother":0,"father":1,"other":2},  # guardian
                        'yes':0,'no':1 
                    }
        )
test(students,0.25,"G3")

MODEL 
           G3
0   1.036005
1  -0.390918
2   -0.04447
3   0.003184
4  -0.160023
5   0.278539
6   0.012595
7   0.114219
8   0.001654
9   0.076725
10  0.175558
11  0.326535
12  0.521019
13 -0.020549
14 -0.080107
15 -0.018036
16 -0.022232
17 -0.143157
18  0.174108
19  0.838774


199.70675252253778